<a href="https://colab.research.google.com/github/Sabri-Nang/analisis_datos/blob/version1/analisis_estadisticas_criminales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datos extraídos de:
[datos.gob.ar](https://www.datos.gob.ar/dataset/seguridad-snic---departamental-estadisticas-criminales-republica-argentina-por-departamentos/archivo/seguridad_3.2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta_csv = '/content/drive/MyDrive/proyectos_análisis_datos/snic-departamentos-anual.csv'

In [ ]:
# importo las librerías
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv(ruta_csv, delimiter=';')

In [ ]:
# muestra
df.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
113243,6,Buenos Aires,6693,Roque Pérez,2003,23,23,Delitos contra el orden público,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
152121,10,Catamarca,10084,Pomán,2022,31,31,Suicidios (consumados),1,1.0,1.0,0.0,0.0,8.570426,8.570426,NaN,NaN
176620,14,Córdoba,14154,Sobremonte,2007,7,7,Lesiones culposas por otros hechos,0,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN
67394,6,Buenos Aires,6392,General Villegas,2014,23,23,Delitos contra el orden público,49,NaN,NaN,NaN,NaN,153.081940,NaN,NaN,NaN
119108,6,Buenos Aires,6735,San Antonio de Areco,2012,29,29,Otros delitos previstos en leyes especiales,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497037 entries, 0 to 497036
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   provincia_id               497037 non-null  int64  
 1   provincia_nombre           497037 non-null  object 
 2   departamento_id            497037 non-null  int64  
 3   departamento_nombre        497037 non-null  object 
 4   anio                       497037 non-null  int64  
 5   codigo_delito_snic_id      497037 non-null  object 
 6   cod_delito                 497037 non-null  int64  
 7   codigo_delito_snic_nombre  497037 non-null  object 
 8   cantidad_hechos            497037 non-null  int64  
 9   cantidad_victimas          176027 non-null  float64
 10  cantidad_victimas_masc     176027 non-null  float64
 11  cantidad_victimas_fem      176027 non-null  float64
 12  cantidad_victimas_sd       176027 non-null  float64
 13  tasa_hechos                47

In [ ]:
df.shape

(497037, 17)

In [ ]:
df.columns

Index(['provincia_id', 'provincia_nombre', 'departamento_id',
       'departamento_nombre', 'anio', 'codigo_delito_snic_id', 'cod_delito',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [ ]:
# veamos si codigo_delito_snic_id	 es igual a cod_delito

df['codigo_delito_snic_id'].astype(pd.Int64Dtype()).isnull().sum()

0

In [ ]:
(df['codigo_delito_snic_id'].astype(pd.Int64Dtype()) != df['cod_delito']).sum() # ambas columnas son iguales

0

In [ ]:
len(df['cod_delito'].unique()) # cantidad de codigos distintos

72

In [ ]:
len(df['codigo_delito_snic_nombre'].unique())

72

In [ ]:
# quiero ver si cada cod_delito corresponde a un codigo_delito_snic_nombre, o hay mas de uno para el mismo codigo
df.groupby('cod_delito')['codigo_delito_snic_nombre'].nunique().reset_index()

,cod_delito,codigo_delito_snic_nombre
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
67,295,2
68,296,2
69,297,1
70,298,2


In [ ]:
# veamos los casos que un codigo engloba mas de un delito
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()

In [ ]:
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)

In [ ]:
codigos_delitos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
0,1,[Homicidios dolosos],1
1,2,[Homicidios dolosos en grado de tentativa],1
2,3,[Muertes en accidentes viales],1
3,4,[Homicidios culposos por otros hechos],1
4,5,[Lesiones dolosas],1
...,...,...,...
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
69,297,[Contrabando de elementos nucleares agresivos ...,1
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [ ]:
distintos = codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]
distintos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
16,17,[Robos agravados por el resultado de lesiones ...,2
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [ ]:
# hay diferencias en los formatos, espacios, mayusculas
# formateo sacando espacios y cambiando todo a minuscula
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.strip()

In [ ]:
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.lower()

In [ ]:
# verifico
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
70,298,"[otros delitos previstos en leyes especiales, ...",2


In [ ]:
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]['codigo_delito_snic_nombre'].iloc[0]

array(['Robos agravados por el resultado de lesiones y/o muertes ',
       'Robos agravados por el resultado de lesiones y/o muertes'],
      dtype=object)

In [ ]:
# reemplazo 'otros delitos previstos en leyes especiales n.c.p' por 'otros delitos previstos en leyes especiales'
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].replace('otros delitos previstos en leyes especiales n.c.p', 'otros delitos previstos en leyes especiales')

In [ ]:
# distintas provincias
print(len(df['provincia_nombre'].unique()))
print(len(df['provincia_id'].unique()))

24
24


In [ ]:
df['provincia_nombre'].unique()

array(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires', 'Catamarca',
       'Córdoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre Ríos',
       'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones',
       'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis',
       'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur'],
      dtype=object)

In [ ]:
# como variaron los delitos por año
# agrupo por año y delito
df.groupby(['anio', 'cod_delito', 'codigo_delito_snic_nombre']).aggregate({'cantidad_hechos': 'sum'})

cantidad_hechos
anio cod_delito codigo_delito_snic_nombre                                          
2000 1          homicidios dolosos                                             2653
     2          homicidios dolosos en grado de tentativa                        884
     3          muertes en accidentes viales                                   3686
     4          homicidios culposos por otros hechos                           1058
     5          lesiones dolosas                                             107864
...                                                                             ...
2023 295        contrabando simple                                            17898
     296        contrabando agravado                                            448
     297        contrabando de elementos nucleares agresivos qu...                7
     298        otros delitos previstos en leyes especiales                   40778
     2810       otros delitos previstos en la ley 23.737                       7432

[938 rows x 1 columns]

# Analizo los homicidios

In [ ]:
df.columns

Index(['provincia_id', 'provincia_nombre', 'departamento_id',
       'departamento_nombre', 'anio', 'codigo_delito_snic_id', 'cod_delito',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [ ]:
df['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos',
       'abusos sexuales con acceso carnal (violaciones)',
       'otros delitos contra la integridad sexual',
       'delitos contra el estado civil', 'amenazas',
       'otros delitos contra la libertad',
       'robos (excluye los agravados por el resultado de lesiones y/o muertes)',
       'tentativas de robo (excluye las agravadas por el res. de lesiones y/o muerte)',
       'robos agravados por el resultado de lesiones y/o muertes',
       'tentativas de robo agravado por el resultado de lesiones y/o muertes',
       'hurtos', 'homicidios dolosos en grado de tentativa',
       'tentativas de hurto', 'otros delitos contra la propiedad',
       'delitos contra la seguridad pública',
       'delitos contra el orden público',
       'delitos contra la seguridad de la nación',
       'delitos contra los poderes públicos y el orden constitucional',
       'delitos contra la administración pública',
       'delitos contra la fe pública', 'ley 23.737 (estupefa

In [ ]:
# cuantos contienen la palabra homicidio
df[df['codigo_delito_snic_nombre'].str.contains('homicidio')]['codigo_delito_snic_nombre']

,codigo_delito_snic_nombre
0,homicidios dolosos
11,homicidios dolosos en grado de tentativa
25,homicidios culposos por otros hechos
31,homicidios dolosos
42,homicidios dolosos en grado de tentativa
...,...
496928,homicidios dolosos en grado de tentativa
496965,homicidios culposos por otros hechos
496971,homicidios dolosos
496988,homicidios dolosos en grado de tentativa


In [ ]:
df_homicidios = df[df['codigo_delito_snic_nombre'].str.contains('homicidio')]
df_homicidios.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
484358,90,Tucumán,90056,Leales,2011,1,1,homicidios dolosos,1,1.0,0.0,1.0,0.0,1.748191,1.748191,0.000000,3.555303
279049,38,Jujuy,38063,San Pedro,2011,4,4,homicidios culposos por otros hechos,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25617,6,Buenos Aires,6084,Benito Juárez,2016,4,4,homicidios culposos por otros hechos,1,1.0,1.0,0.0,0.0,4.879715,4.879715,9.620935,0.000000
164496,14,Córdoba,14063,Marcos Juárez,2006,4,4,homicidios culposos por otros hechos,3,3.0,3.0,0.0,0.0,2.899475,2.899475,NaN,NaN
431155,78,Santa Cruz,78049,Río Chico,2012,1,1,homicidios dolosos,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [ ]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38379 entries, 0 to 497031
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   provincia_id               38379 non-null  int64  
 1   provincia_nombre           38379 non-null  object 
 2   departamento_id            38379 non-null  int64  
 3   departamento_nombre        38379 non-null  object 
 4   anio                       38379 non-null  int64  
 5   codigo_delito_snic_id      38379 non-null  object 
 6   cod_delito                 38379 non-null  int64  
 7   codigo_delito_snic_nombre  38379 non-null  object 
 8   cantidad_hechos            38379 non-null  int64  
 9   cantidad_victimas          38379 non-null  float64
 10  cantidad_victimas_masc     38379 non-null  float64
 11  cantidad_victimas_fem      38379 non-null  float64
 12  cantidad_victimas_sd       38379 non-null  float64
 13  tasa_hechos                36648 non-null  float64

In [ ]:
# tipos de delitos
df_homicidios['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos', 'homicidios dolosos en grado de tentativa',
       'homicidios culposos por otros hechos'], dtype=object)

In [ ]:
hechos_provincia = df_homicidios.pivot_table(
    columns="provincia_nombre",
    index="anio",
    values="tasa_victimas",
    aggfunc="sum"
)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.bar(data_frame=hechos_provincia)

fig.show()

In [ ]:
fig = px.scatter(data_frame=hechos_provincia)
fig.show()

In [ ]:
hechos_provincia.columns

Index(['Buenos Aires', 'Catamarca', 'Chaco', 'Chubut',
       'Ciudad Autónoma de Buenos Aires', 'Corrientes', 'Córdoba',
       'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza',
       'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis',
       'Santa Cruz', 'Santa Fe', 'Santiago del Estero',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur', 'Tucumán'],
      dtype='object', name='provincia_nombre')

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Crear la figura con una matriz de 8x3
fig = make_subplots(rows=8, cols=3, subplot_titles=hechos_provincia.columns)

# Añadir cada gráfico de barra a la matriz
y_axis_range = [0, hechos_provincia.values.max()]
for i, provincia in enumerate(hechos_provincia.columns):
    row = (i // 3) + 1
    col = (i % 3) + 1
    subfig = px.bar(data_frame=hechos_provincia, y=provincia)

    # Añadir la traza
    fig.add_trace(
        subfig.data[0],
        row=row, col=col
    )

    # Configurar los ejes y el título de cada subgráfico
    fig.update_yaxes(range=y_axis_range, row=row, col=col)
    #fig.update_xaxes(title_text=provincia, row=row, col=col)  # Opcional: Añadir título al eje x

# Añadir títulos a cada subgráfico
#for i, title in enumerate(hechos_provincia.columns):
    fig.layout.annotations[i].update(text=provincia, font=dict(size=12))

fig.add_annotation(
    text="Año",
    xref="paper", yref="paper",
    x=0.5, y=-0.1,  # Ajusta la posición según sea necesario
    showarrow=False,
    font=dict(size=16)
)
fig.add_annotation(
    text="Tasa de homicidios",  # Texto para el eje Y
    xref="paper", yref="paper",
    x=-0.08, y=0.5,  # Ajusta la posición según sea necesario
    showarrow=False,
    textangle=-90,  # Rotar el texto 90 grados para que sea vertical
    font=dict(size=16)
)
fig.update_layout(
    height=1000, width=1100,
    showlegend=True,
    title_text="Gráficos de Hechos por Provincia",
    title_x=0.5,
    #title_y=0.95,

)

fig.show()
